## **GBM (Classification)**

In [ ]:
# 필요한 기본 package 불러오기
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import matplotlib

from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import time
import warnings
warnings.filterwarnings('ignore')

In [ ]:
!git clone https://github.com/KU-DIC/LG_time_series_day07.git

### **(1) 마케팅 성공 여부 예측**

In [ ]:
'''
Bank Marketing Data : 포르투칼 은행의 전화 마케팅 데이터, 전화 마케팅의 성공 여부를 고객의 개인정보를 이용해 예측
'''

# 데이터 불러오기
csv = pd.read_csv("/content/LG_time_series_day07/Data_GBM_bank-full.csv", encoding = "UTF-8-sig", sep = ";")
csv.head(10)

In [ ]:
# 데이터 타입(type) 확인
csv.dtypes

In [ ]:
#전처리 된 데이터 만들기
'''
연속형 변수 정수형 -> 실수형
범주형 변수 -> Dummie Coding
'''

data = pd.DataFrame()

for i, j in enumerate(csv.iloc[:, : -1].dtypes.items()): #데이터의 컬럼과 해당 컬럼의 데이터 타입을 for 문에 입력
    #해당 컬럼의 데이터 타입이 연속형(정수형)일 경우,
    if j[1] == "int64":
        #실수형으로 바꾼 컬럼을 data 변수에 추가
        data = pd.concat([data, csv.iloc[:, i].astype(float)], axis = 1, sort = False)
    #해당 컬럼의 데이터 타입이 범주형일 경우,
    else:
        #Dummies 코딩 수행
        dummies = pd.get_dummies(csv.iloc[:, i])
        dummies.columns = [j[0] + "_" + k for k in dummies.columns]
        #Dummie 코딩한 컬럼을 data에 저장
        data = pd.concat([data, dummies], axis = 1, sort = False)

#종속 변수 "yes"와 "no"를 실수형으로 형태 변경
data.loc[csv.y == "yes", "y"] = 1.0
data.loc[csv.y == "no", "y"] = -1.0
data.y = data.y.astype(float)

In [ ]:
# 변경된 데이터 확인 (첫 10개의 instances)
data.head(10)

In [ ]:
# 학습 데이터와 테스트 데이터 구분
# 학습 데이터 비율: 0.7, 테스트 데이터 비율: 0.3
train_data, test_data = train_test_split(data, train_size = 0.7)

# 독립변수(Xs)와 종속변수(Y) 구분
train_X = train_data.iloc[:, :-1].reset_index(drop = True) # train_X에 종속변수 제거
train_Y = train_data.iloc[:, -1].reset_index(drop = True) # train_Y에 종속변수 따로 저장

test_X = test_data.iloc[:, :-1].reset_index(drop = True) # test_X에 종속변수 제거
test_Y = test_data.iloc[:, -1].reset_index(drop = True) # test_Y에 종속변수 따로 저장

In [ ]:
# GBM 모델(분류) package 불러오기
from sklearn.ensemble import GradientBoostingClassifier

# 모델 파라미터 설정
model = GradientBoostingClassifier(loss          = "deviance",
                                   learning_rate = 0.1,
                                   n_estimators  = 100,
                                   criterion     = "mse",
                                   max_depth     = 3,
                                   min_samples_split = 2,
                                   min_samples_leaf  = 1,
                                   verbose = 1)

# 설정된 모델 파라미터에 데이터 fitting (학습)
model.fit(train_X, train_Y)

In [ ]:
# 학습한 GBM 모델을 통해 테스트 데이터 예측하기

# pred 변수에 실제값(y) 추가
pred = pd.DataFrame(test_Y)

# 생성된 모델로 예측하기 / pred 변수에 예측값 추가
pred["pred"] = model.predict(test_X)

# 예측 값 확인 (첫 10개 instances)
pred.head(10)

In [ ]:
# Confusion Matrix 생성
tab = pd.crosstab(pred.y, pred.pred)

#정확도 구하기
acc = (tab.iloc[0,0] + tab.iloc[1,1]) / len(test_Y)

print("Confusion Matrix")
print(tab)
print("   ")
print("Acc : ", acc)

### **(2) 가입자 이탈 여부 예측**

In [ ]:
'''
가입자 이탈/이탈률 dataset : 전체 19개 피처와 1개의 타겟으로 구성 (Categorical feature: State, Area_code, Int.l.Plan, VMail.Plan)

타겟명: 'Churn.' / True(1)는 이탈한 고객, False(0)는 이탈하지 않은 고객을 의미함
'''

# 데이터 불러오기
churn = pd.read_csv("/content/LG_time_series_day07/Data_GBM_churn.csv")
print('dataset shape:', churn.shape)

# 데이터 확인 (첫 3개 instances)
churn.head(3)

In [ ]:
# 데이터 타입(type) 확인
churn.info()

In [ ]:
# 데이터 전처리

# column name 중 "."을 "_"으로 변경
churn.columns = churn.columns.str.replace(".","_")

# data중 Area.Code는 지역숫자로 수치적 의미를 갖지 않으므로, category 형태로 변경
churn.Area_Code = churn.Area_Code.astype("category")

print(churn['Churn_'].value_counts())

unsatisfied_cnt = churn[churn['Churn_'] == "True." ].Churn_.count()
total_cnt = churn.Churn_.count()

print('\n\n unsatisfied 비율은 {0:.3f}'.format((unsatisfied_cnt / total_cnt)))

In [ ]:
# 변경된 데이터 타입(type) 확인
churn.info()

In [ ]:
# 독립변수(Xs)와 종속변수(Y) 구분
X_features = churn.iloc[:, :-1]
y_labels = churn.iloc[:, -1]

# 타겟 변수에 대해 불만족한 고객 ('True.')을 1로, 나머지는 0으로 해서 최종적으로 binary class {1, 0}로 변경
y_labels = np.where(y_labels == "True.", 1, 0)
y_labels = pd.Series(y_labels)

print('feature shape:{0}'.format(X_features.shape))

In [ ]:
# object 타입을 category로 변경
for col in X_features.columns:
    if X_features[col].dtypes == "O":
        X_features[col] = X_features[col].astype('category')

# 데이터 타입(type) 확인
X_features.info()

In [ ]:
# GBM에서 categorical feature에 대해 one-hot encoding 진행
def onehot(data, feature):
    return pd.concat([data, pd.get_dummies(data[feature], prefix=feature)], axis=1).drop([feature], axis=1)

# 다음 4가지 categorical feature에 대해 one-hot encoding을 진행 후 기존 X_feature에 결합
cate_list = ['State', 'Area_Code', 'Int_l_Plan', 'VMail_Plan']
for i in range(len(cate_list)):
    X_features = onehot(X_features, cate_list[i])

In [ ]:
# 추가된 feature 확인
X_features.info()

In [ ]:
# 학습 데이터와 테스트 데이터 구분
# 학습 데이터 비율: 0.8, 테스트 데이터 비율: 0.2
X_train, X_test, y_train, y_test = train_test_split(X_features, y_labels, test_size=0.2, random_state=0)
train_cnt = y_train.count()
test_cnt = y_test.count()
print('학습 세트 Shape:{0}, 테스트 세트 Shape:{1}'.format(X_train.shape , X_test.shape))

print(' 학습 세트 레이블 값 분포 비율')
print(y_train.value_counts()/train_cnt)
print('\n 테스트 세트 레이블 값 분포 비율')
print(y_test.value_counts()/test_cnt)

In [ ]:
# GBM 모델 package 불러오기
from sklearn.ensemble import GradientBoostingClassifier

# GBM 수행 시간 측정을 위함. 시작 시간 설정
start_time = time.time()

# 모델 생성 및 train data로 fitting (학습) 진행
gb_clf = GradientBoostingClassifier(random_state=0)
gb_clf.fit(X_train , y_train)

# GBM 실험 종료에 대한 시간 측정
gb_time = time.time() - start_time

In [ ]:
# 학습 완료된 GBM 모델을 통해 테스트 데이터 예측
gb_pred = gb_clf.predict(X_test)

# 예측 값과 실제 값 간의 비교를 통해, 정확도(차이) 계산
gb_accuracy = accuracy_score(y_test, gb_pred)

print('GBM 정확도: {0:.4f}'.format(gb_accuracy))
print("GBM 수행 시간: {0:.2f} 초 ".format(gb_time))

gb_roc_score = roc_auc_score(y_test, gb_clf.predict_proba(X_test)[:,1], average='macro')
print('ROC AUC: {0:.4f}'.format(gb_roc_score))

In [ ]:
# 최적 hyper parameter를 찾기 위해 GridSearchCV 실행
from sklearn.model_selection import GridSearchCV

params = {
    'n_estimators':[100, 500],
    'learning_rate' : [ 0.05, 0.1]
}

gb_grid_cv = GridSearchCV(gb_clf , param_grid=params , cv=2 ,verbose=1, n_jobs=-1)
gb_grid_cv.fit(X_train , y_train)
print('최적 하이퍼 파라미터:\n', gb_grid_cv.best_params_)
print('최고 예측 정확도: {0:.4f}'.format(gb_grid_cv.best_score_))

In [ ]:
# GridSearchCV를 이용하여 최적으로 학습된 estimator로 예측 수행
gb_pred = gb_grid_cv.best_estimator_.predict(X_test)
gb_accuracy = accuracy_score(y_test, gb_pred)
print('GBM 정확도: {0:.4f}'.format(gb_accuracy))

gb_grid_cv_roc_score = roc_auc_score(y_test, gb_grid_cv.predict_proba(X_test)[:,1],average='macro')
print('GBM ROC AUC: {0:.4f}'.format(gb_grid_cv_roc_score))